In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [40]:
data = pd.read_csv("BTC-USD_SIGNALS.csv")

In [41]:
data

,Date,Open,High,Low,Close,Volume,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Confirmation Time,Transactions,Miners Revenue,FnG
0,2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,-0.292146,-2.712213,-1.043523,-1.108359,-0.475907,1.319161,-0.323408,0.521410,1.144750,-0.774692,-0.186892,-0.583209
1,2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,-0.152676,-1.011481,-1.204172,-1.202571,-0.654534,1.402518,-0.323408,0.393472,1.144750,-0.774692,-0.186892,-1.244407
2,2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,-0.427993,0.955604,-1.632900,-1.235294,-0.618233,-0.269198,0.287977,0.409785,1.114249,-0.774692,-0.186892,-0.142410
3,2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,-0.437581,-2.584947,-1.626830,-1.320379,-0.741794,0.093536,-0.323408,0.427573,1.114249,-1.144752,-0.264496,-0.847688
4,2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,-0.326118,-4.184569,-1.814295,-1.479953,-1.057732,1.828350,-0.323408,0.462483,1.114249,-1.144752,-0.264496,-1.420727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2022-09-04,19832.470703,19999.689453,19636.816406,19986.712891,0.478227,0.148948,-1.113287,-0.844382,-0.254845,-0.167922,-0.323408,0.699748,-1.063067,-0.648298,-0.307825,-1.024008
1680,2022-09-05,19988.789063,20031.160156,19673.046875,19812.371094,0.658018,-0.278482,-1.133325,-0.831832,-0.166540,-0.269198,-0.256709,0.691814,-0.742360,-0.648298,-0.307825,-0.891768
1681,2022-09-06,19817.724609,20155.269531,18800.171875,18837.667969,1.393318,-1.325705,-1.548973,-0.892284,-0.308652,1.167318,-0.323408,0.776149,-0.742360,-0.165203,-0.359746,-0.935848
1682,2022-09-07,18837.683594,19427.171875,18644.466797,19290.324219,0.981876,0.568952,-1.180050,-0.893278,-0.249058,-0.012950,-0.323408,0.881401,-0.742360,-0.165203,-0.359746,-0.847688


In [42]:
predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [43]:
predict_data = predict_data[[i % int(max_days / 2) == 0 for i in range(len(predict_data))]]

In [44]:
predict_data

,Date,Volume,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Confirmation Time,Transactions,Miners Revenue,FnG,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,CONF-1,TRANS-1,REV-1,FnG-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,CONF-2,TRANS-2,REV-2,FnG-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,CONF-3,TRANS-3,REV-3,FnG-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,CONF-4,TRANS-4,REV-4,FnG-4,Target
0,2018-02-05,-0.326118,-4.184569,-1.814295,-1.479953,-1.057732,1.828350,-0.323408,0.462483,1.114249,-1.144752,-0.264496,-1.420727,-2.584947,-0.437581,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688,0.955604,-0.427993,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410,-1.011481,-0.152676,-1.204172,-1.202571,-0.654534,1.144750,-0.774692,-0.186892,-1.244407,-2.712213,-0.292146,-1.043523,-1.108359,-0.475907,1.144750,-0.774692,-0.186892,-0.583209,1
2,2018-02-07,-0.331965,-0.495589,-1.457407,-1.552132,-0.860381,-0.269198,1.072032,0.525668,0.453384,-1.959889,-0.596734,-0.318730,2.918547,-0.088527,-1.212491,-1.523793,-0.973190,0.453384,-1.144752,-0.264496,-1.552966,-4.184569,-0.326118,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,-2.584947,-0.437581,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688,0.955604,-0.427993,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410,1
4,2018-02-09,-0.452131,1.422827,-0.830140,-1.412981,-0.171285,-0.269198,0.054294,0.626783,0.207605,-1.959889,-0.596734,0.033910,2.134561,-0.323021,-1.063630,-1.504877,-0.549827,0.453384,-1.959889,-0.596734,-0.583209,-0.495589,-0.331965,-1.457407,-1.552132,-0.860381,0.453384,-1.959889,-0.596734,-0.318730,2.918547,-0.088527,-1.212491,-1.523793,-0.973190,0.453384,-1.144752,-0.264496,-1.552966,-4.184569,-0.326118,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,0
6,2018-02-11,-0.485525,-1.529055,-1.178450,-1.298888,0.178688,0.330456,-0.323408,0.655642,0.207605,-2.453448,-0.455877,-0.539129,-0.393589,-0.401930,-0.947564,-1.334806,0.091563,0.207605,-2.453448,-0.455877,0.474709,1.422827,-0.452131,-0.830140,-1.412981,-0.171285,0.207605,-1.959889,-0.596734,0.033910,2.134561,-0.323021,-1.063630,-1.504877,-0.549827,0.453384,-1.959889,-0.596734,-0.583209,-0.495589,-0.331965,-1.457407,-1.552132,-0.860381,0.453384,-1.959889,-0.596734,-0.318730,1
8,2018-02-13,-0.506967,-1.004265,-0.584168,-1.121564,0.570097,-0.269198,-0.323408,0.722799,0.276565,-1.522212,-0.160496,-0.362809,2.482440,-0.478760,-0.746790,-1.191801,0.455801,0.276565,-2.453448,-0.455877,-0.054250,-1.529055,-0.485525,-1.178450,-1.298888,0.178688,0.207605,-2.453448,-0.455877,-0.539129,-0.393589,-0.401930,-0.947564,-1.334806,0.091563,0.207605,-2.453448,-0.455877,0.474709,1.422827,-0.452131,-0.830140,-1.412981,-0.171285,0.207605,-1.959889,-0.596734,0.033910,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,2022-08-28,0.433926,-0.601366,-2.100264,-0.799658,-1.290428,0.181253,-0.323408,-0.074029,-1.263537,-1.414496,-0.174982,-0.671369,-0.331552,0.723698,-2.049936,-0.689661,-1.213271,-1.263537,-0.310257,-0.097684,-0.671369,-1.658930,1.339033,-2.000380,-0.584031,-1.132733,-1.025714,-0.310257,-0.097684,-0.715449,0.196202,0.769656,-1.385521,-0.464083,-0.979903,-1.025714,-0.310257,-0.097684,-0.803608,-0.212717,0.816704,-1.545836,-0.444016,-1.152727,-1.025714,-0.498097,-0.206563,-0.803608,1
1670,2022-08-30,0.943757,-0.691652,-1.653641,-0.871357,-1.020465,-0.269198,0.069068,0.238574,-1.313931,-1.414496,-0.174982,-0.715449,0.845674,0.850751,-1.597787,-0.821959,-1.096574,-1.263537,-1.414496,-0.174982,-0.847688,-0.601366,0.433926,-2.100264,-0.799658,-1.290428,-1.263537,-1.414496,-0.174982,-0.671369,-0.331552,0.723698,-2.049936,-0.689661,-1.213271,-1.263537,-0.310257,-0.097684,-0.671369,-1.658930,1.339033,-2.000380,-0.584031,-1.132733,-1.025714,-0.310257,-0.097684,-0.715449,1
1672,2022-09-01,0

In [45]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date","Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date","Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date","Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [48]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [49]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(200,2000,200),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Iteration 1, loss = 0.75101587
Iteration 2, loss = 0.69023775
Iteration 3, loss = 0.66252296
Iteration 4, loss = 0.64346398
Iteration 5, loss = 0.62759941
Iteration 6, loss = 0.60726810
Iteration 7, loss = 0.58357481
Iteration 8, loss = 0.56351119
Iteration 9, loss = 0.54456601
Iteration 10, loss = 0.52373617
Iteration 11, loss = 0.47815434
Iteration 12, loss = 0.45968312
Iteration 13, loss = 0.42683962
Iteration 14, loss = 0.39388749
Iteration 15, loss = 0.35376035
Iteration 16, loss = 0.32010218
Iteration 17, loss = 0.28718691
Iteration 18, loss = 0.25470753
Iteration 19, loss = 0.23305351
Iteration 20, loss = 0.19373705
Iteration 21, loss = 0.18939781
Iteration 22, loss = 0.22755299
Iteration 23, loss = 0.15900613
Iteration 24, loss = 0.13473910
Iteration 25, loss = 0.12978465
Iteration 26, loss = 0.12690233
Iteration 27, loss = 0.11534901
Iteration 28, loss = 0.08571650
Iteration 29, loss = 0.07297928
Iteration 30, loss = 0.05928956
Iteration 31, loss = 0.04919931
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(200, 2000, 200), verbose=True)

In [50]:
model.score(test_data,test_labels)

0.5018050541516246

In [51]:
model.score(train_data,train_labels)

1.0

In [52]:
from sklearn.metrics import confusion_matrix

In [53]:
preds = model.predict(test_data)

In [54]:
conf = confusion_matrix(test_labels,preds)
conf

array([[53, 72],
       [66, 86]], dtype=int64)

In [55]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.44537815126050423

In [56]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5443037974683544

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date","Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


### Using TensorFlow

In [21]:
%load_ext tensorboard

In [22]:
!rm -rf ./logs/

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [72]:
shape = (len(predict_data.columns)-2,)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [73]:
print(train_data.shape, test_data.shape)

(562, 48) (277, 48)


In [74]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [75]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [76]:
model.fit(train_data, train_labels, epochs=30, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback])

Epoch 1/30
18/18 [==============================] - 4s 142ms/step - loss: 0.7538 - accuracy: 0.5320 - precision: 0.5391 - recall: 0.8087 - val_loss: 0.6873 - val_accuracy: 0.5487 - val_precision: 0.5487 - val_recall: 1.0000
Epoch 2/30
18/18 [==============================] - 2s 102ms/step - loss: 0.6805 - accuracy: 0.5302 - precision: 0.5304 - recall: 0.9966 - val_loss: 0.6878 - val_accuracy: 0.5668 - val_precision: 0.5769 - val_recall: 0.7895
Epoch 3/30
18/18 [==============================] - 2s 103ms/step - loss: 0.6691 - accuracy: 0.5765 - precision: 0.5743 - recall: 0.7785 - val_loss: 0.6927 - val_accuracy: 0.5307 - val_precision: 0.5775 - val_recall: 0.5395
Epoch 4/30
18/18 [==============================] - 2s 111ms/step - loss: 0.6651 - accuracy: 0.6139 - precision: 0.6421 - recall: 0.6141 - val_loss: 0.7008 - val_accuracy: 0.5523 - val_precision: 0.5854 - val_recall: 0.6316
Epoch 5/30
18/18 [==============================] - 2s 104ms/step - loss: 0.6455 - accuracy: 0.6423 - pr

In [28]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

4/4 - 0s - loss: 3.1906 - accuracy: 0.4414 - precision: 0.5116 - recall: 0.3492 - 28ms/epoch - 7ms/step


ValueError: too many values to unpack (expected 2)

In [147]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



In [148]:
preds = np.argmax(probability_model.predict(test_data),axis=1)

In [149]:
conf = confusion_matrix(test_labels,preds)
conf

array([[51, 74],
       [62, 90]], dtype=int64)

In [150]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.45132743362831856

In [151]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5487804878048781